Use Gradient Tree Boosting to predict monthly stock returns

Report:
1. The respective columns you calc in Table 1 of Gu et al (2020)
2. The respective columns in Tables 5 and A.7 of Gu et al (2020)

You will need FF3 and FF5 porfolio breakpoiints and weights for Table 5 and A.7. Available in the course bucket

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb

In [2]:
data_dir = '/Users/anishkahc/Desktop/Docs/ML Fall 23/Data/'

openap_file = 'OpenAP_Macro.parquet.gzip'
openap = pd.read_parquet( data_dir + openap_file)

#Code from HW03
# filter data from year 2000 onwards
openap = openap[pd.DatetimeIndex(openap['DateYM']).year >= 2000]
openap.drop(openap.iloc[:,-10:-1], inplace = True, axis = 1)
openap.set_index(keys = ['DateYM', 'permno'], inplace = True, verify_integrity = True)
#rank
cols = openap.columns[:-1]
openap[cols] = openap.groupby('DateYM')[cols].rank(pct=True).subtract(0.5).multiply(2)

In [3]:
#missing dataset replaced by relevant period median value code from HW03
def fill_missing_values(x):
    median_value = x.median()
    return x.fillna(0 if pd.isna(median_value) else median_value)

openap = openap.groupby('DateYM').transform(fill_missing_values)
openap.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1570968 entries, (Timestamp('2000-01-31 00:00:00'), 10001) to (Timestamp('2020-12-31 00:00:00'), 93436)
Data columns (total 207 columns):
 #    Column                     Non-Null Count    Dtype  
---   ------                     --------------    -----  
 0    AbnormalAccruals           1570968 non-null  float64
 1    Accruals                   1570968 non-null  float64
 2    AccrualsBM                 1570968 non-null  float64
 3    Activism1                  1570968 non-null  float64
 4    Activism2                  1570968 non-null  float64
 5    AdExp                      1570968 non-null  float64
 6    AgeIPO                     1570968 non-null  float64
 7    AM                         1570968 non-null  float64
 8    AnalystRevision            1570968 non-null  float64
 9    AnalystValue               1570968 non-null  float64
 10   AnnouncementReturn         1570968 non-null  float64
 11   AOP                        1570968 non-

In [4]:
# def split_val(openap):
#     return model_selection.PredefinedSplit(openap['test_fold'])

In [5]:
def R2_oos(y, y_pred):
    mse = 0
    mse_benchmark = 0
    for i in range(len(y)):
        mse += (y[i] - y_pred[i])**2
        mse_benchmark += y[i]**2
    R2 = 1 - mse/mse_benchmark
    return R2

In [24]:
year=[]
r2_result = []
predictions_df = pd.DataFrame(columns=['permno', 'DateYM', 'Y_true', 'Y_pred'])

n = 10

train_start = 2000
train_end = 2005
validation_end = 2008
test_end = 2009
    
for i in range(n):
    print("Training year ", i + 2009) 
    year.append(i + 2009)

    train_val = openap[(openap.index.get_level_values('DateYM').year >= train_start) 
                       & (openap.index.get_level_values('DateYM').year <= validation_end+i)].copy()
    test = openap[(openap.index.get_level_values('DateYM').year > validation_end+i) 
                  & (openap.index.get_level_values('DateYM').year <= test_end+i)].copy()
    
    train_val.sort_index(inplace=True)
    test.sort_index(inplace=True)
        
    train_val = train_val.assign(test_fold = -1)
    train_val.loc[pd.DatetimeIndex(train_val.index.get_level_values('DateYM')).year > train_end+i, 'test_fold'] = 0
        
    xgb_train_val = xgb.XGBRegressor(n_estimators=100, 
                                       max_depth=3, 
                                       learning_rate=0.05,
                                       n_jobs=4
                                    )
    
    X_train_val = train_val.drop(columns=['test_fold', 'retadj'])
    Y_train_val = train_val['retadj']
    
    xgb_train_val.fit(X_train_val, Y_train_val)
    
    Y_train = xgb_train_val.predict(X=X_train_val)
    Y_test = xgb_train_val.predict(X=test.drop(columns=['retadj']))
    
    # Calculate R^2
    r2_train = R2_oos(Y_train_val, Y_train)
    print('Training R2:', r2_train)
    
    r2_test = metrics.r2_score(test['retadj'], Y_test)
    print('Test R2:', r2_test)
    
    r2_result.append((i+2009, r2_train, r2_test))
    
    #Referred to Ashutosh's code for saving the prediction values
    predictions = pd.DataFrame({
        'permno': test.index.get_level_values('permno'),
        'DateYM': test.index.get_level_values('DateYM'),
        'y_true': test['retadj'],
        'y_pred': Y_test,
    })
    
#     print(type(predictions))
#     print(type(predict_df))
#     print(pd.__version__)
#     print(r2_result)
    predictions_df = pd.concat([predictions_df,predictions], ignore_index=True)  


Training year  2009
Training R2: 0.12677856720103897
Test R2: -0.25049311596006807
Training year  2010
Training R2: 0.12391231780600676
Test R2: -0.03340363323724227
Training year  2011
Training R2: 0.12216733988876594
Test R2: -0.013262894383752055
Training year  2012
Training R2: 0.11760536026850121
Test R2: -0.03944444050098994
Training year  2013
Training R2: 0.11220965925832738
Test R2: -0.0536901243511434
Training year  2014
Training R2: 0.11016561075689502
Test R2: -0.001924146663529136
Training year  2015
Training R2: 0.10605745076750006
Test R2: -0.010098518505927467
Training year  2016
Training R2: 0.09893934639407387
Test R2: -0.06791888494130482
Training year  2017
Training R2: 0.09623899023696347
Test R2: -0.018098821289378453
Training year  2018
Training R2: 0.09139888912683891
Test R2: -0.005415578656037479


### Columns for Table 5 and A.7

In [21]:
#Data Preprocessing 

# Referred to Ashutosh's code from HW04 as I faced issues during merging for my HW04. 
# His function makes the code more efficient as well.  

def load_and_preprocess_data(filename):
    data = pd.read_parquet(data_dir + filename)
    # Data for year 2000 and after
    data = data[pd.DatetimeIndex(data['jdate']).year >= 2000]
    data.rename(columns={'jdate': 'DateYM'}, inplace=True)
    return data

#Load data
CMA_ret = load_and_preprocess_data("Gu-Table5/CMA_ret.parquet.gzip")
CMA_wt = load_and_preprocess_data("Gu-Table5/CMA_wt.parquet.gzip")

FF3_ret = load_and_preprocess_data("Gu-Table5/FF-3_ret.parquet.gzip")
FF3_wt = load_and_preprocess_data("Gu-Table5/FF-3_wt.parquet.gzip")

RMW_ret = load_and_preprocess_data("Gu-Table5/RMW_ret.parquet.gzip")
RMW_wt = load_and_preprocess_data("Gu-Table5/RMW_wt.parquet.gzip")

SP500_ret = load_and_preprocess_data("Gu-Table5/SP500_VW_ret.parquet.gzip")
SP500_wt = load_and_preprocess_data("Gu-Table5/SP500_VW_wt.parquet.gzip")

UMD_ret = load_and_preprocess_data("Gu-Table5/UMD_ret.parquet.gzip")
UMD_wt = load_and_preprocess_data("Gu-Table5/UMD_wt.parquet.gzip")

In [22]:
CMA_ret

,DateYM,port,ret
3060,2000-01-31,BA,-0.057577
3061,2000-01-31,BC,-0.007716
3062,2000-01-31,BM,-0.043734
3063,2000-01-31,SA,-0.005573
3064,2000-01-31,SC,0.004742
...,...,...,...
4219,2018-05-31,SM,0.067947
4220,2018-06-30,BA,0.005457
4221,2018-06-30,BM,0.005063
4222,2018-06-30,SA,0.011524


In [ ]:
# # wt_ret is weighted_return
# def calc_wt_ret(predictions_df, wt_df):
#     # merging the return and wt dataframes on 'jdate'
#     merged_df = pd.merge(predictions_df, wt_df, on=['permno', 'DateYM'], how='left') 
    
#     # Calculating weighted return
#     merged_df['wt_return'] = merged_df['ret'] * merged_df['port_wt']
#     merged_df['Year'] = pd.DatetimeIndex(merged_df['DateYM']).year
#     merged_df = merged_df.groupby('DateYM')['wt_return'].sum().reset_index()
#     return merged_df

In [27]:
# Using CMA
CMA_merged = CMA_wt.merge(predictions_df, on=['permno', 'DateYM'], how='left')
CMA_merged['wt_ypred'] = CMA_merged['y_pred'] * CMA_merged['port_wt']

result_CMA = CMA_merged.groupby(['DateYM', 'port'])['wt_ypred'].sum().reset_index()
CMA_merged['Year'] = pd.DatetimeIndex(CMA_merged['DateYM']).year
result_CMA = CMA_merged.groupby(['Year', 'port'])['wt_ypred'].sum().reset_index()

CMA_ret['DateYM'] = pd.to_datetime(CMA_ret['DateYM'])
CMA_ret['Year'] = CMA_ret['DateYM'].dt.year
CMA_yearly_ret = CMA_ret.groupby(['port', 'Year'])['ret'].mean().reset_index()

CMA_merged_ret = CMA_yearly_ret.merge(result_CMA, on=['Year', 'port'], how='inner')
CMA_merged_ret['Sharpe_Ratio'] = (CMA_merged_ret['ret'] - 0.045) / CMA_merged_ret['wt_ypred']
CMA_merged_ret['R^2'] = (CMA_merged_ret['ret'] - CMA_merged_ret['wt_ypred'])**2
CMA_merged_ret

,port,Year,ret,wt_ypred,Sharpe_Ratio,R^2
0,BA,2000,-0.020069,0.000000,-inf,0.000403
1,BA,2001,-0.015525,0.000000,-inf,0.000241
2,BA,2002,-0.024494,0.000000,-inf,0.000600
3,BA,2003,0.020975,0.000000,-inf,0.000440
4,BA,2004,0.010610,0.000000,-inf,0.000113
...,...,...,...,...,...,...
99,SM,2014,0.005541,0.015403,-2.561723,0.000097
100,SM,2015,-0.004962,0.084784,-0.589290,0.008054
101,SM,2016,0.020503,-0.053770,0.455595,0.005516
102,SM,2017,0.009238,0.232440,-0.153856,0.049819


In [28]:
# Using FF3
FF3_merged = FF3_wt.merge(predictions_df, on=['permno', 'DateYM'], how='left')
FF3_merged['wt_ypred'] = FF3_merged['y_pred'] * FF3_merged['port_wt']

result_FF3 = FF3_merged.groupby(['DateYM', 'port'])['wt_ypred'].sum().reset_index()
FF3_merged['Year'] = pd.DatetimeIndex(FF3_merged['DateYM']).year
result_FF3 = FF3_merged.groupby(['Year', 'port'])['wt_ypred'].sum().reset_index()

FF3_ret['DateYM'] = pd.to_datetime(FF3_ret['DateYM'])
FF3_ret['Year'] = FF3_ret['DateYM'].dt.year
FF3_yearly_ret = FF3_ret.groupby(['port', 'Year'])['ret'].mean().reset_index()

FF3_merged_ret = FF3_yearly_ret.merge(result_FF3, on=['Year', 'port'], how='inner')
FF3_merged_ret['Sharpe_Ratio'] = (FF3_merged_ret['ret'] - 0.045) / FF3_merged_ret['wt_ypred']
FF3_merged_ret['R^2'] = (FF3_merged_ret['ret'] - FF3_merged_ret['wt_ypred'])**2
FF3_merged_ret

,port,Year,ret,wt_ypred,Sharpe_Ratio,R^2
0,BH,2000,0.022489,0.000000,-inf,0.000506
1,BH,2001,0.003011,0.000000,-inf,0.000009
2,BH,2002,-0.025044,0.000000,-inf,0.000627
3,BH,2003,0.021197,0.000000,-inf,0.000449
4,BH,2004,0.016133,0.000000,-inf,0.000260
...,...,...,...,...,...,...
109,SM,2014,0.006081,0.015201,-2.560213,0.000083
110,SM,2015,-0.001035,0.089223,-0.515950,0.008146
111,SM,2016,0.020209,-0.048073,0.515693,0.004662
112,SM,2017,0.009963,0.235815,-0.148578,0.051009


In [29]:
# Using RMW
RMW_merged = RMW_wt.merge(predictions_df, on=['permno', 'DateYM'], how='left')
RMW_merged['wt_ypred'] = RMW_merged['y_pred'] * RMW_merged['port_wt']

result_RMW = RMW_merged.groupby(['DateYM', 'port'])['wt_ypred'].sum().reset_index()
RMW_merged['Year'] = pd.DatetimeIndex(RMW_merged['DateYM']).year
result_RMW = RMW_merged.groupby(['Year', 'port'])['wt_ypred'].sum().reset_index()

RMW_ret['DateYM'] = pd.to_datetime(RMW_ret['DateYM'])
RMW_ret['Year'] = RMW_ret['DateYM'].dt.year
RMW_yearly_ret = RMW_ret.groupby(['port', 'Year'])['ret'].mean().reset_index()

RMW_merged_ret = RMW_yearly_ret.merge(result_RMW, on=['Year', 'port'], how='inner')
RMW_merged_ret['Sharpe_Ratio'] = (RMW_merged_ret['ret'] - 0.045) / RMW_merged_ret['wt_ypred']
RMW_merged_ret['R^2'] = (RMW_merged_ret['ret'] - RMW_merged_ret['wt_ypred'])**2
RMW_merged_ret

,port,Year,ret,wt_ypred,Sharpe_Ratio,R^2
0,BM,2000,-0.007419,0.000000,-inf,0.000055
1,BM,2001,-0.013967,0.000000,-inf,0.000195
2,BM,2002,-0.019727,0.000000,-inf,0.000389
3,BM,2003,0.026592,0.000000,-inf,0.000707
4,BM,2004,0.009265,0.000000,-inf,0.000086
...,...,...,...,...,...,...
109,SW,2014,0.005976,0.008995,-4.338435,0.000009
110,SW,2015,-0.002026,0.081534,-0.576770,0.006982
111,SW,2016,0.019912,-0.063264,0.396567,0.006918
112,SW,2017,0.009304,0.228482,-0.156229,0.048039


In [30]:
# Using UMD
UMD_merged = UMD_wt.merge(predictions_df, on=['permno', 'DateYM'], how='left')
UMD_merged['wt_ypred'] = UMD_merged['y_pred'] * UMD_merged['port_wt']

result_UMD = UMD_merged.groupby(['DateYM', 'port'])['wt_ypred'].sum().reset_index()
UMD_merged['Year'] = pd.DatetimeIndex(UMD_merged['DateYM']).year
result_UMD = UMD_merged.groupby(['Year', 'port'])['wt_ypred'].sum().reset_index()

UMD_ret['DateYM'] = pd.to_datetime(UMD_ret['DateYM'])
UMD_ret['Year'] = UMD_ret['DateYM'].dt.year
UMD_yearly_ret = UMD_ret.groupby(['port', 'Year'])['ret'].mean().reset_index()

UMD_merged_ret = UMD_yearly_ret.merge(result_UMD, on=['Year', 'port'], how='inner')
UMD_merged_ret['Sharpe_Ratio'] = (UMD_merged_ret['ret'] - 0.045) / UMD_merged_ret['wt_ypred']
UMD_merged_ret['R^2'] = (UMD_merged_ret['ret'] - UMD_merged_ret['wt_ypred'])**2
UMD_merged_ret

,port,Year,ret,wt_ypred,Sharpe_Ratio,R^2
0,BD,2003,0.032105,0.000000,-inf,0.001031
1,BD,2004,0.008718,0.000000,-inf,0.000076
2,BD,2005,0.003160,0.000000,-inf,0.000010
3,BD,2006,-0.009059,0.000000,-inf,0.000082
4,BD,2007,0.008822,0.000000,-inf,0.000078
...,...,...,...,...,...,...
97,SU,2014,0.004473,0.014241,-2.845751,0.000095
98,SU,2015,0.001286,0.097297,-0.449286,0.009218
99,SU,2016,0.013249,-0.047516,0.668204,0.003692
100,SU,2017,0.014717,0.238423,-0.127013,0.050044


In [31]:
# using HML
SL = FF3_yearly_ret[FF3_yearly_ret['port'] == 'SL']['ret'].reset_index(drop=True)
SH = FF3_yearly_ret[FF3_yearly_ret['port'] == 'SH']['ret'].reset_index(drop=True)
BL = FF3_yearly_ret[FF3_yearly_ret['port'] == 'BL']['ret'].reset_index(drop=True)
BH = FF3_yearly_ret[FF3_yearly_ret['port'] == 'BH']['ret'].reset_index(drop=True)

SL_ret = FF3_ret[FF3_ret['port'] == 'SL']['wt_ypred'].reset_index(drop=True)
SH_ret = FF3_ret[FF3_ret['port'] == 'SH']['wt_ypred'].reset_index(drop=True)
BL_ret = FF3_ret[FF3_ret['port'] == 'BL']['wt_ypred'].reset_index(drop=True)
BH_ret = FF3_ret[FF3_ret['port'] == 'BH']['wt_ypred'].reset_index(drop=True)


HML = pd.DataFrame()
HML['Year'] = yearly_returns_FF3['Year'].unique()
HML['ret'] = ((SL + BL) / 2 - (SH + BH) / 2).values
HML['ret_pred'] = ((SL_ret + BL_ret) / 2 - (SH_ret + BH_ret) / 2).values
HML['Sharpe_Ratio'] = (HML['ret'] - 0.045) / HML['ret_pred']
HML['R^2'] = (HML['ret'] - HML['ret_pred'])**2
HML

KeyError: 'wt_ypred'